In [1]:

from copy import deepcopy

import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import time

In [2]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [3]:
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=28 * 28, hidden_size=16):
        super(SimpleFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, 10)
        # self.fc1 = nn.Linear(hidden_size, 10)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.fc0(x)
        return x

In [5]:
# Dataset and Dataloader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

# Load dataset and split into train/validation sets
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [6]:
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc0])

In [7]:
hyperparams = {
    "num_epochs": 64,
    "metric": MagnitudeL2Metric(nn.CrossEntropyLoss()), 
    "aggregation_mode": "mean",  
    "choose_threshold": 0.5,  
    "threshold": 0.05,
    "window_size": 5,  
    "lr": 1e-4, 
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

'num_epochs: 64, metric: MagnitudeL2Metric, aggregation_mode: mean, choose_threshold: 0.5, threshold: 0.05, window_size: 5, lr: 0.0001'

In [8]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: fedornigretuk to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
run = wandb.init(
    project="self-expanding-nets",
    name=f"trash",
)

In [10]:
train_sparse_recursive(sparse_model, train_loader, val_loader, hyperparams)

100%|██████████| 750/750 [00:09<00:00, 80.86it/s]


Epoch 1/64, Train Loss: 1.4805, Val Loss: 0.9977, Val Accuracy: 0.8192


100%|██████████| 750/750 [00:08<00:00, 84.28it/s]


Epoch 2/64, Train Loss: 0.8088, Val Loss: 0.6843, Val Accuracy: 0.8528


100%|██████████| 750/750 [00:09<00:00, 79.10it/s]


...

Epoch 12/64, Train Loss: 0.3243, Val Loss: 0.3221, Val Accuracy: 0.9114


100%|██████████| 750/750 [00:13<00:00, 55.44it/s]


Epoch 13/64, Train Loss: 0.3142, Val Loss: 0.3138, Val Accuracy: 0.9129


  5%|▌         | 39/750 [00:00<00:12, 54.87it/s]


KeyboardInterrupt: 

- прунинг по метрике на следующей эпохе после реплейса
